In [22]:
from pymatgen.ext.matproj import MPRester
from pymatgen.io.cif import CifWriter
from pymatgen.electronic_structure.bandstructure import BandStructure
from pymatgen.electronic_structure.plotter import BSPlotter
from numpy import *
from scipy import constants
import matplotlib as plt

    # --------------------------------------------------------
    # Set-up.
    # --------------------------------------------------------

with MPRester("L41ZHCirMyRASy8QHJEO") as m:
    
    structure = m.get_structure_by_material_id("mp-22526")
    
    BSPresult = BSPlotter(BandStructure)
    BSPresult.show()
    Tab = BSPresult.bs_plot_data(zero_to_efermi=True)
    
    Distances = Tab['distances']
    Energies  = Tab['energy']
    
    Index = BandStructure.get_vbm(band_structure)["band_index"]
    print(Index)
    
    # --------------------------------------------------------
    # Obtention du gap.
    # --------------------------------------------------------

    Gap = BandStructure.get_band_gap()
    
    print("========== Get_band_gap_value ==========\n")
    print("Informations sur la bande interdite : \n")
    if (Gap.get('direct') == True):
        print("Bande directe\n")
    else:
        print("Bande indirecte\n")
    print("Energie de gap : ",Gap.get('energy'),"\n")
    print("Vecteur d'onde de transition : ",Gap.get('transition'),"\n")
    print("========== Done ==========\n")
    
    # --------------------------------------------------------
    # Calcul de la dispersion.
    # --------------------------------------------------------
    
    # @return     : indice de la branche pour les conditions spécifiées
    # @parameters : extremum = 'min' : permet d'obtenir la branche à la plus faible pente.
    #                        = 'max' : permet d'obtenir la branche à la plus forte pente.
    #               BandeValence = True : calculs effectués pour la bande de valence.
    #                            = False : calculs effectués pour la bande de conduction.
    
    def Dispersion(extremum,BandeValence):
        result = -1
        Z = Index # Indice correspondant au vbm
        if(BandeValence == False):
            Z += 1 #Indice correspondant au cbm = vbm + 1
        
        if(extremum == 'max'):
            Tol = -1 # Utilisé pour indiquer quand on trouve une valeur plus grande qu'une précédente.
            for i in range(0,len(Distances)):
                E = Energies[i]['1'][Z]
                D = Distances[i]
                Pente = (E[-1] - E[0]) / (D[-1] - [0])
                if (abs(Pente) > Tol):
                    Tol = Pente
                    result = i
            return result
        if(extremum == 'min'):
            Tol = 15 # Utilisé pour indiquer quand on trouve une valeur plus petite qu'une précédente.
            for i in range(0,len(Distances)):
                E = Energies[i]['1'][Z]
                D = Distances[i]
                Pente = (E[-1] - E[0]) / (D[-1] - [0])
                if (abs(Pente) < Tol):
                    Tol = Pente
                    result = i
            return result
        print("Error : incorrect extremum specification !\n") # Normalement impossible.
        print(extremum)
        return result
    # --------------------------------------------------------
    
    plt.figure("-1-")
    BSPresult.get_plot(ylim=[-3,8])
    
    # ---------- Minimum de dispersion de la bande de valence ----------
    
    indicesValence1 = Dispersion('min',True)
    # Indication du point à gauche et à droite.
    plt.scatter([Distances[indicesValence1][0]],Energies[indicesValence1]['1'][Index][0],c='blue')
    plt.scatter([Distances[indicesValence1][-1]],Energies[indicesValence1]['1'][Index][-1],c='blue')
    # Coordonnées de début et de fin de la flèche.
    x0,x1 = Distances[indicesValence1][0],Distances[indicesValence1][-1]
    y0,y1 = Energies[indicesValence1]['1'][Index][0],Energies[indicesValence1]['1'][Index][-1]
    # Ajout de la flèche à la figure.
    plt.arrow(x1,y1,x0-x1,y0-y1,head_width=0.1,head_length=0.1,length_includes_head=True,color='blue')
    
    # ---------- Maximum de dispersion de la bande de valence ----------
    
    indicesValence2 = Dispersion('max',True)
    # Indication du point à gauche et à droite.
    plt.scatter([Distances[indicesValence2][0]],Energies[indicesValence2]['1'][Index][0],c='red')
    plt.scatter([Distances[indicesValence2][-1]],Energies[indicesValence2]['1'][Index][-1],c='red')
    # Coordonnées de début et de fin de la flèche.
    x0,x1 = Distances[indicesValence2][0],Distances[indicesValence2][-1]
    y0,y1 = Energies[indicesValence2]['1'][Index][0],Energies[indicesValence2]['1'][Index][-1]
    # Ajout de la flèche à la figure.
    plt.arrow(x1,y1,x0-x1,y0-y1,head_width=0.1,head_length=0.1,length_includes_head=True,color='red')
    
    # ---------- Minimum de dispersion de la bande de conduction ----------
    
    indicesConduction1 = Dispersion('min',False)
    # Indication du point à gauche et à droite.
    plt.scatter([Distances[indicesConduction1][0]],Energies[indicesConduction1]['1'][Index+1][0],c='blue')
    plt.scatter([Distances[indicesConduction1][-1]],Energies[indicesConduction1]['1'][Index+1][-1],c='blue')
    # Coordonnées de début et de fin de la flèche.
    x0,x1 = Distances[indicesConduction1][0],Distances[indicesConduction1][-1]
    y0,y1 = Energies[indicesConduction1]['1'][Index+1][0],Energies[indicesConduction1]['1'][Index+1][-1]
    # Ajout de la flèche à la figure.
    plt.arrow(x1,y1,x0-x1,y0-y1,head_width=0.1,head_length=0.1,length_includes_head=True,color='blue')
    
    # ---------- Maximum de dispersion de la bande de conduction ----------
    
    indicesConduction2 = Dispersion('max',False)
    # Indication du point à gauche et à droite.
    plt.scatter([Distances[indicesConduction2][0]],Energies[indicesConduction2]['1'][Index+1][0],c='red')
    plt.scatter([Distances[indicesConduction2][-1]],Energies[indicesConduction2]['1'][Index+1][-1],c='red')
    # Coordonnées de début et de fin de la flèche.
    x0,x1 = Distances[indicesConduction2][0],Distances[indicesConduction2][-1]
    y0,y1 = Energies[indicesConduction2]['1'][Index+1][0],Energies[indicesConduction2]['1'][Index+1][-1]
    # Ajout de la flèche à la figure.
    plt.arrow(x1,y1,x0-x1,y0-y1,head_width=0.1,head_length=0.1,length_includes_head=True,color='red')
    
    # ---------- Affichage ----------

    plt.show()
    print("===== Done =====\n")
    
    
    # --------------------------------------------------------
    # Calcul de la masse effective.
    # --------------------------------------------------------
    
    plt.figure('-2-')
    BSPresult.get_plot()
    
    # ----- Pour le maximum de bande de valence -----
    
    E_maxV = Energies[indicesValence2]['1'][Index]
    D_maxV = Distances[indicesValence2]
    
    CoefPoly_maxV = polyfit(D_maxV,E_maxV,2) # coefficients de la parabole.
    xPlot_maxV = linspace(D_maxV - 1.,E_maxV + 1., 1000) # input parabole.
    yPlot_maxV = polyval(CoefPoly_maxV,xPlot_maxV) # output parabole.
    
    # ----- Pour le minimum de bande de conduction -----
    
    E_minC = Energies[indicesConduction1]['1'][Index+1]
    D_minC = Distances[indicesConduction1]
    
    CoefPoly_minC = polyfit(D_minC,E_minC,2) # coefficients de la parabole.
    xPlot_minC = linspace(D_maxV - 0.5,E_maxV + 1., 1000) # input parabole.
    yPlot_minC = polyval(CoefPoly_minC,xPlot_minC) # output parabole.
    
    # ----- Calculs -----
    
    h = constants.hbar # constante de Planck / 2pi
    a = 1.602e-19 # angstrom
    
    masseEffective_vide       = 9.109e-31 # Masse effective de l'électron dans le vide
    masseEffective_valence    = (h**2) / (2*CoefPoly_maxV[0]) / (1e-20 * a)
    masseEffective_conduction = (h**2) / (2*CoefPoly_minC[0]) / (1e-20 * a)
    
    # ----- Affichage -----
    
    print("===== Rapport des masses effectives avec la masse de l'électron dans le vide =====\n")
    print(" Bande de valence : ", masseEffective_valence/masseEffective_vide,"\n")
    print("Bande de conduction :", masseEffective_conduction/masseEffective_vide,"\n")
    
    plt.plot(xPlot_maxV,yPlot_maxV,'-b')
    plt.plot(xPlot_minC,yPlot_minC,'-r')
    plt.show()
    
    print("\n===== Done =====\n")
    
    # --------------------------------------------------------
    # Fin de tâche.
    # --------------------------------------------------------

ValueError: BSPlotter only works with BandStructureSymmLine objects. A BandStructure object (on a uniform grid for instance and not along symmetry lines won't work)